<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/70_Vision_Transformer_ViT.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 👁️ Vision Transformer (ViT): Obraz jest wart 16x16 słów

CNN ma tzw. **Inductive Bias** (zakłada z góry, że lokalność jest ważna).
Transformer nie zakłada niczego. Uczy się relacji między pikselem w lewym górnym rogu, a pikselem w prawym dolnym rogu od razu (Global Attention).

**Algorytm ViT:**
1.  **Patching:** Podziel obrazek na kwadraty (np. 16x16 pikseli).
2.  **Flatten:** Spłaszcz każdy kwadrat do wektora. To są nasze "słowa".
3.  **Position Embedding:** Dodaj informację, gdzie ten kwadrat był na obrazku (użyjemy trenowalnych parametrów, a nie sinusa).
4.  **CLS Token:** Dodaj jeden specjalny, pusty wektor na początku. Po przejściu przez sieć, to on będzie zawierał informację "Co jest na obrazku?".
5.  **Transformer Encoder:** Standardowe bloki (Attention + MLP).

Użyjemy zbioru CIFAR-10 (obrazki 32x32). Podzielimy je na Patche 4x4.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Konfiguracja
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 64
IMG_SIZE = 32
PATCH_SIZE = 4
EMBED_DIM = 128  # Rozmiar wektora, w jaki zamienimy patch
NUM_HEADS = 4
NUM_LAYERS = 4
NUM_CLASSES = 10

# Sprawdźmy matematykę
num_patches = (IMG_SIZE // PATCH_SIZE) ** 2
input_dim = 3 * PATCH_SIZE * PATCH_SIZE # 3 kanały RGB * 4 * 4 piksele

print(f"Obrazek: {IMG_SIZE}x{IMG_SIZE}")
print(f"Patch: {PATCH_SIZE}x{PATCH_SIZE}")
print(f"Liczba Patchy: {num_patches} (To będzie długość naszego 'zdania')")
print(f"Wymiar jednego Patcha (spłaszczony): {input_dim}")

Obrazek: 32x32
Patch: 4x4
Liczba Patchy: 64 (To będzie długość naszego 'zdania')
Wymiar jednego Patcha (spłaszczony): 48


## Krok 1: Patch Embedding Layer

Musimy zamienić obrazek (3D) na sekwencję wektorów (2D).
Moglibyśmy użyć pętli i ciąć obrazek, ale jest sprytniejszy sposób.

**Inżynierski Trik:**
Użycie warstwy `Conv2d` z rozmiarem kernela równym `patch_size` i krokiem (`stride`) równym `patch_size` robi dokładnie to samo! Tnie obrazek na kawałki i od razu rzutuje je na wymiar `EMBED_DIM`.

In [2]:
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=3, patch_size=4, emb_size=128, img_size=32):
        super().__init__()
        self.patch_size = patch_size
        
        # Trik: Conv2d jako tokenizer
        self.projection = nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size)
        
    def forward(self, x):
        # x: [Batch, C, H, W] -> [Batch, Emb_Size, H/P, W/P]
        x = self.projection(x)
        
        # Spłaszczamy: [Batch, Emb_Size, Num_Patches]
        x = x.flatten(2)
        
        # Transpozycja: [Batch, Num_Patches, Emb_Size] (Tak lubią Transformery)
        x = x.transpose(1, 2)
        return x

# Test
dummy_img = torch.randn(1, 3, 32, 32)
pe = PatchEmbedding(patch_size=PATCH_SIZE, emb_size=EMBED_DIM)
out = pe(dummy_img)
print(f"Wejście: {dummy_img.shape}")
print(f"Wyjście (Sekwencja): {out.shape} -> [Batch, Liczba Patchy, Wymiar]")

Wejście: torch.Size([1, 3, 32, 32])
Wyjście (Sekwencja): torch.Size([1, 64, 128]) -> [Batch, Liczba Patchy, Wymiar]


## Krok 2: The ViT (Składamy całość)

Tutaj dzieje się magia.
1.  Tworzymy **CLS Token** (learnable parameter). Doklejamy go na początek sekwencji.
2.  Tworzymy **Position Embeddings** (też learnable). Dodajemy je do sekwencji.
3.  Przepuszczamy przez **Transformer Encoder** (PyTorch ma gotowy moduł `nn.TransformerEncoder`, ale to jest dokładnie to, co budowaliśmy w notatniku 46).
4.  Na końcu bierzemy TYLKO pierwszy token (CLS) i na jego podstawie klasyfikujemy obraz.

In [3]:
class VisionTransformer(nn.Module):
    def __init__(self, embed_dim, n_heads, n_layers, num_classes, patch_size, img_size):
        super().__init__()
        
        # 1. Tokenizacja (Patching)
        self.patch_embed = PatchEmbedding(patch_size=patch_size, emb_size=embed_dim, img_size=img_size)
        
        # Liczba patchy
        self.num_patches = (img_size // patch_size) ** 2
        
        # 2. Token CLS (Klasyfikacyjny) - parametryzowany wektor
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        
        # 3. Pozycje (Learnable) - dla wszystkich patchy + 1 (CLS)
        self.pos_embed = nn.Parameter(torch.randn(1, 1 + self.num_patches, embed_dim))
        
        # 4. Transformer Encoder (Stack bloków)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=n_heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # 5. Głowica klasyfikująca (MLP Head)
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x):
        # x: [Batch, 3, 32, 32]
        
        # Embeddings
        x = self.patch_embed(x)
        
        # Doklejamy CLS Token
        batch_size = x.shape[0]
        cls_tokens = self.cls_token.expand(batch_size, -1, -1) # Kopiujemy dla całego batcha
        x = torch.cat((cls_tokens, x), dim=1) # [Batch, N+1, Dim]
        
        # Dodajemy pozycje
        x = x + self.pos_embed
        
        # Transformer
        x = self.transformer(x)
        
        # Klasyfikacja: Bierzemy tylko token 0 (CLS)
        cls_output = x[:, 0]
        
        return self.mlp_head(cls_output)

# Inicjalizacja modelu
model = VisionTransformer(
    embed_dim=EMBED_DIM,
    n_heads=NUM_HEADS,
    n_layers=NUM_LAYERS,
    num_classes=NUM_CLASSES,
    patch_size=PATCH_SIZE,
    img_size=IMG_SIZE
).to(DEVICE)

print(f"ViT gotowy. Liczba parametrów: {sum(p.numel() for p in model.parameters()):,}")

ViT gotowy. Liczba parametrów: 2,388,362


In [4]:
# POBIERANIE DANYCH (CIFAR-10)
# Obrazki: Samolot, Auto, Ptak, Kot...
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# TRENING (Szybki test - 2 epoki)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("Start treningu ViT...")
model.train()

for epoch in range(2): # Tylko 2 epoki, bo Transformer jest wolny na CPU
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        if batch_idx % 100 == 0:
            print(f"Epoka {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}")

print("✅ Trening zakończony.")

Start treningu ViT...
Epoka 0, Batch 0, Loss: 2.4581
Epoka 0, Batch 100, Loss: 2.1943
Epoka 0, Batch 200, Loss: 1.9984
Epoka 0, Batch 300, Loss: 2.1356
Epoka 0, Batch 400, Loss: 2.0505
Epoka 0, Batch 500, Loss: 2.1221
Epoka 0, Batch 600, Loss: 1.7942
Epoka 0, Batch 700, Loss: 1.8145
Epoka 1, Batch 0, Loss: 1.8641
Epoka 1, Batch 100, Loss: 2.0023
Epoka 1, Batch 200, Loss: 1.9983
Epoka 1, Batch 300, Loss: 1.8797
Epoka 1, Batch 400, Loss: 1.9642
Epoka 1, Batch 500, Loss: 2.0154
Epoka 1, Batch 600, Loss: 1.9771
Epoka 1, Batch 700, Loss: 1.8348
✅ Trening zakończony.


## 🧠 Podsumowanie: ViT vs CNN

Co zauważysz?
ViT prawdopodobnie uczy się **wolniej** lub daje gorsze wyniki na małym zbiorze (CIFAR-10) niż proste CNN.

**Dlaczego?**
*   **CNN** ma "wrodzoną wiedzę" (Inductive Bias): wie, że piksele obok siebie są ważne. To pomaga przy małych danych.
*   **ViT** musi się wszystkiego nauczyć od zera (nawet tego, co to znaczy "być obok siebie").

**Ale...**
Gdy dasz ViT-owi 300 milionów zdjęć (zbiór JFT-300M), ViT miażdży CNN. CNN dochodzi do szklanego sufitu, a ViT skaluje się w nieskończoność. Dlatego GPT-4V, Gemini i DALL-E używają Transformerów do obrazu.